<a href="https://colab.research.google.com/github/annakalinina18/star-fle/blob/main/annotation_avec_LLM/gpt_5_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from openai import OpenAI
import pandas as pd
from tqdm import tqdm

client = OpenAI(api_key="")

In [ ]:
# =========================
# 1. НАСТРОЙКА МОДЕЛИ
# =========================

# Более мощная модель (НЕ mini)
MODEL_NAME = "gpt-5.1"


# =========================
# 2. PROMPTS POUR CHAQUE TEST
# =========================

CRAN_PROMPT = """
Tu es linguiste annotateur. Réponds UNIQUEMENT par "OUI" ou "NON".

TEST CRAN — mot exclusivement figé
Question : l’expression contient-elle un mot exclusivement figé ?

Un mot exclusivement figé :
- n’existe pas comme mot autonome,
- n’apparaît jamais en dehors de l’expression,
- n’a plus d’usage productif moderne.

Exemple OUI : « us » dans « us et coutumes »
Exemple NON : « bleu » dans « cordon bleu »

Réponds uniquement OUI ou NON.
Puis ajoute UNE phrase très courte expliquant la raison.
Format obligatoire :
OUI/NON
Explication : <phrase très courte>

Expression : {expression}
Contexte : {contexte}
"""

MORPH_PROMPT = """
Tu es linguiste annotateur. Réponds UNIQUEMENT par "OUI" ou "NON".

TEST MORPH — modification morphologique
Question : la flexion (pluriel, genre) rend-elle l’expression agrammaticale ou étrange ?

Exemple OUI : « garde du corps » → « gardes du corps »
Exemple NON : « livre scolaire » → « livres scolaires »

Réponds uniquement OUI ou NON.
Puis ajoute UNE phrase très courte expliquant la raison.
Format obligatoire :
OUI/NON
Explication : <phrase très courte>

Expression : {expression}
Contexte : {contexte}
"""

SYNT_PROMPT = """
Tu es linguiste annotateur. Réponds UNIQUEMENT par "OUI" ou "NON".

TEST SYNT — modification syntaxique
Question : un changement d’ordre ou de structure modifie-t-il fortement le sens ou le rend-il agrammaticale ou bizzare ?

Exemple OUI : « pomme de terre » ≠ « pomme terrestre »
Exemple NON : « comité régional » ≈ « comité de la région »

Réponds uniquement OUI ou NON.
Puis ajoute UNE phrase très courte expliquant la raison.
Format obligatoire :
OUI/NON
Explication : <phrase très courte>

Expression : {expression}
Contexte : {contexte}
"""

LEX_PROMPT = """
Tu es linguiste annotateur. Réponds UNIQUEMENT par "OUI" ou "NON".

TEST LEX — substitution lexicale
Question : remplacer un élément par un synonyme rend-il l’expression anormale, agrammaticale ou bizzare ?

Exemple OUI : « eau de vie » → « boisson de vie »
Exemple NON : « prix réduit » → « tarif réduit »

Réponds uniquement OUI ou NON.
Puis ajoute UNE phrase très courte expliquant la raison.
Format obligatoire :
OUI/NON
Explication : <phrase très courte>

Expression : {expression}
Contexte : {contexte}
"""

MODIF_PROMPT = """
Tu es linguiste annotateur. Réponds UNIQUEMENT par "OUI" ou "NON".

TEST MODIF — insertion d’un modifieur
Question : l’insertion d’un modifieur (adjectif/adverbe) interne rend-elle l’expression anormale, agrammaticale ou bizzare ?

Exemple OUI : « pomme verte de terre »
Exemple NON : « roman contemporain policier »

Réponds uniquement OUI ou NON.
Puis ajoute UNE phrase très courte expliquant la raison.
Format obligatoire :
OUI/NON
Explication : <phrase très courte>

Expression : {expression}
Contexte : {contexte}
"""

SEMREST_PROMPT = """
Tu es linguiste annotateur. Réponds UNIQUEMENT par "OUI" ou "NON".

TEST SEM-REST — restrictions sémantiques
Question : la tête nominale (le nom principal de l'expression) telle qu'elle est utilisée dans cette expression (dans son sens contextuel) a-t-elle un ensemble restreint de collocatifs naturels ?

Exemple OUI : « professeur de linguistique » → mais aussi « professeur de maths », « professeur de chimie », « professeur de littérature » (ensemble restreint aux domaines académiques)
Exemple NON : « événement majeur » → mais aussi « événement local », « événement imprévu », « événement politique », etc. (ensemble très ouvert)

⚠️ Attention : Tu dois analyser UNIQUEMENT la tête nominale, pas l’expression entière.
On vérifie avec quels collocatifs naturels se combine la tête nominale seule.

Exemples :
- dans « roman policier » → la tête nominale = « roman », pas « roman policier »
- dans « pomme de terre » → la tête nominale = « pomme », pas « pomme de terre »

Ne prends jamais l’expression complète comme base de comparaison.
Toujours la tête nominale seule.

⚠️ Attention : Dans ces données, il est rare que la réponse à ce test est NON.

Réponds uniquement OUI ou NON.
Puis ajoute UNE phrase très courte expliquant la raison.
Format obligatoire :
OUI/NON
Explication : <phrase très courte>

Expression : {expression}
Contexte : {contexte}
"""

ID_PROMPT = """
Tu es linguiste annotateur. Réponds UNIQUEMENT par "OUI" ou "NON".

TEST ID — type naturel
Question : l’expression désigne-t-elle une sous-catégorie naturelle et littérale du noyau nominal ?


⚠️ Attention : tu dois identifier la tête syntaxique de l’expression :
le nom qui constitue le noyau de la construction nominale. L’objectif du test est de déterminer si l’expression désigne une sous-catégorie
naturelle de ce noyau.

Pour répondre OUI, le noyau nominal doit garder son sens et l’expression doit désigner un type de ce nom. La réponse est plus souvent OUI que NON, donc dans les cas d'incertitude penche vers OUI.

Exemples ID = NON :
- « île flottante » (dessert) : ce n’est pas un type d'île, c'est un dessert
- « forêt noire » (gâteau) : ce n’est pas un type de forêt, c'est un gâteau

Exemple ID = OUI :
- « événement culturel » : c’est un type d’événement
- « chaise en bois » : c’est un type de chaise
- « véhicule électrique » : c’est un type de véhicule
- « bouteille en plastique » : c’est un type de bouteille
- « porte coulissante » : c’est un type de porte



Réponds uniquement OUI ou NON.
Puis ajoute UNE phrase très courte expliquant la raison.
Format obligatoire :
OUI/NON
Explication : <phrase très courte>

Expression : {expression}
Contexte : {contexte}
"""

OPAC_PROMPT = """
Tu es linguiste annotateur. Réponds UNIQUEMENT par "OUI" ou "NON".

🔵 TEST OPAC — opacité sémantique

Question : le sens global ne correspond-il pas à la somme des sens littéraux des composants ?

Si le noyau nominal n’est pas littéral → OPAC = OUI.

Exemples OUI :
- « île flottante » (pas une île flotte)
- « forêt noire » (pas une forêt qui est noire)
- « lune de miel » (pas une lune construite de miel)

Exemples NON :
- « analyse des données » (c'est une analyse impliquant des données)
- « décision administrative » (c'est une décision qui est de type administratif)

Expression : « {expression} »
Contexte : {contexte}

Réponds uniquement OUI ou NON.
Puis ajoute UNE phrase très courte expliquant la raison.
Format obligatoire :
OUI/NON
Explication : <phrase très courte>

Expression : {expression}
Contexte : {contexte}
"""


# =========================
# 3. ВСПОМОГАТЕЛЬНАЯ ФУНКЦИЯ ДЛЯ ОДНОГО ТЕСТА
# =========================

def ask_test(prompt_template, expression, examples):
    contexte = "" if examples is None or pd.isna(examples) else str(examples)
    prompt = prompt_template.format(expression=expression, contexte=contexte)

    resp = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[{"role": "user", "content": prompt}],
        max_completion_tokens=50,
    )

    full = (resp.choices[0].message.content or "").strip()

    # первая строка = ответ OUI/NON
    first_line = full.split("\n")[0].strip().upper()

    decision = "OUI" if first_line == "OUI" else "NON"

    # объяснение = всё, что идёт после первой строки
    explanation = ""
    if "\n" in full:
        explanation = full.split("\n", 1)[1].strip()

    return decision, explanation


# =========================
# 4. ДЕРЕВО РЕШЕНИЙ ДЛЯ ВСЕХ ТЕСТОВ (ОДИН ПРОГОН)
# =========================

def annotate_expression(expression, examples, debug=False):
    tests = {}

    # CRAN
    d, e = ask_test(CRAN_PROMPT, expression, examples)
    tests["CRAN"] = (d, e)
    if debug:
        print("CRAN:", d, e)
    if d == "OUI":
        return tests, "expression_idiomatique"

    # MORPH
    d, e = ask_test(MORPH_PROMPT, expression, examples)
    tests["MORPH"] = (d, e)
    if debug:
        print("MORPH:", d, e)
    if d == "OUI":
        d2, e2 = ask_test(ID_PROMPT, expression, examples)
        tests["ID"] = (d2, e2)
        if debug:
            print("ID (après MORPH):", d2, e2)
        if d2 == "NON":
            return tests, "expression_idiomatique"
        d3, e3 = ask_test(OPAC_PROMPT, expression, examples)
        tests["OPAC"] = (d3, e3)
        if debug:
            print("OPAC (après MORPH+ID):", d3, e3)
        return tests, ("collocation_opaque" if d3 == "OUI" else "collocation_transparente")

    # SYNT
    d, e = ask_test(SYNT_PROMPT, expression, examples)
    tests["SYNT"] = (d, e)
    if debug:
        print("SYNT:", d, e)
    if d == "OUI":
        d2, e2 = ask_test(ID_PROMPT, expression, examples)
        tests["ID"] = (d2, e2)
        if debug:
            print("ID (après SYNT):", d2, e2)
        if d2 == "NON":
            return tests, "expression_idiomatique"
        d3, e3 = ask_test(OPAC_PROMPT, expression, examples)
        tests["OPAC"] = (d3, e3)
        if debug:
            print("OPAC (après SYNT+ID):", d3, e3)
        return tests, ("collocation_opaque" if d3 == "OUI" else "collocation_transparente")

    # LEX
    d, e = ask_test(LEX_PROMPT, expression, examples)
    tests["LEX"] = (d, e)
    if debug:
        print("LEX:", d, e)
    if d == "OUI":
        d2, e2 = ask_test(ID_PROMPT, expression, examples)
        tests["ID"] = (d2, e2)
        if debug:
            print("ID (après LEX):", d2, e2)
        if d2 == "NON":
            return tests, "expression_idiomatique"
        d3, e3 = ask_test(OPAC_PROMPT, expression, examples)
        tests["OPAC"] = (d3, e3)
        if debug:
            print("OPAC (après LEX+ID):", d3, e3)
        return tests, ("collocation_opaque" if d3 == "OUI" else "collocation_transparente")

    # MODIF
    d, e = ask_test(MODIF_PROMPT, expression, examples)
    tests["MODIF"] = (d, e)
    if debug:
        print("MODIF:", d, e)
    if d == "OUI":
        d2, e2 = ask_test(ID_PROMPT, expression, examples)
        tests["ID"] = (d2, e2)
        if debug:
            print("ID (après MODIF):", d2, e2)
        if d2 == "NON":
            return tests, "expression_idiomatique"
        d3, e3 = ask_test(OPAC_PROMPT, expression, examples)
        tests["OPAC"] = (d3, e3)
        if debug:
            print("OPAC (après MODIF+ID):", d3, e3)
        return tests, ("collocation_opaque" if d3 == "OUI" else "collocation_transparente")

    # SEM-REST
    d, e = ask_test(SEMREST_PROMPT, expression, examples)
    tests["SEM-REST"] = (d, e)
    if debug:
        print("SEM-REST:", d, e)
    if d == "NON":
        return tests, "expression_libre"

    # ID
    d, e = ask_test(ID_PROMPT, expression, examples)
    tests["ID"] = (d, e)
    if debug:
        print("ID (après SEM-REST):", d, e)
    if d == "NON":
        return tests, "expression_idiomatique"

    # OPAC
    d, e = ask_test(OPAC_PROMPT, expression, examples)
    tests["OPAC"] = (d, e)
    if debug:
        print("OPAC (fin):", d, e)
    return tests, ("collocation_opaque" if d == "OUI" else "collocation_transparente")


# =========================
# 5. ОБВЁРТКА ДЛЯ ЗАПИСИ В DATAFRAME (БЕЗ MAJORITY VOTE)
# =========================

def generate_annotation_with_llm(expression, examples, debug=False):
    tests, category = annotate_expression(expression, examples, debug=debug)

    lines = []

    # Имитация "Votes :" (один голос — финальная категория)
    lines.append("Votes :")
    lines.append(f"- {category}")

    # Финальная категория
    lines.append(f"\nCATÉGORIE FINALE : {category}")

    # Объяснения тестов
    if tests:
        lines.append("\nExplications des tests menant à la catégorie finale :")
        order = ["CRAN", "MORPH", "SYNT", "LEX", "MODIF", "SEM-REST", "ID", "OPAC"]
        for t in order:
            if t in tests:
                decision, expl = tests[t]
                lines.append(f"{t} : {decision}")
                if expl:
                    lines.append(f"  Explication : {expl}")

    text = "\n".join(lines)

    if debug:
        print("\n=== RÉSUMÉ (un seul run, explications) ===\n")
        print(text)

    return text


# =========================
# 6. ОБРАБОТКА EXCEL
# =========================

chunk_file_path = "nominal_part_7.xlsx"
df_chunk = pd.read_excel(chunk_file_path)

df_chunk["llm_raw_response"] = None

for index, row in tqdm(
    df_chunk.iterrows(),
    total=len(df_chunk),
    desc="Annotation des expressions nominales",
    unit="expr",
):
    expression = row.get("expression")
    examples = row.get("examples_joined")

    if pd.isna(expression) or str(expression).strip() == "":
        df_chunk.at[index, "llm_raw_response"] = "N/A (missing expression)"
        continue

    debug_flag = index < 1  # дебаг только для первой строки

    llm_answer = generate_annotation_with_llm(
        expression=expression,
        examples=examples,
        debug=debug_flag,
    )

    df_chunk.at[index, "llm_raw_response"] = llm_answer

output_filename = "annotated_nominal_part_7_gpt51_single_run.xlsx"
df_chunk.to_excel(output_filename, index=False)
print(f"Annotated and saved {output_filename}")


Annotation des expressions nominales:   0%|          | 0/100 [00:00<?, ?expr/s]

CRAN: NON Explication : Les mots « système » et « phonologique » sont autonomes et productifs en français.
MORPH: NON Explication : Le pluriel « systèmes phonologiques » reste grammatical et naturel.
SYNT: NON Explication : L’ordre inverse « phonologique du système » reste compréhensible et grammatical.
LEX: NON Explication : « système phonologique » peut être remplacé par « système des sons » sans bizarrerie.
MODIF: NON Explication : L’adjectif « phonologique » modifie normalement « système » ici.
SEM-REST: OUI Explication : « système » combine avec des ensembles typés mais limités (phonologique, solaire, nerveux, éducatif, etc.).
ID (après SEM-REST): OUI Explication : C’est un type particulier de système.


Annotation des expressions nominales:   1%|          | 1/100 [00:10<17:04, 10.35s/expr]

OPAC (fin): NON Explication : C’est bien un système constitué des phonèmes/sons d’une langue, sens littéral du noyau.

=== RÉSUMÉ (un seul run, explications) ===

Votes :
- collocation_transparente

CATÉGORIE FINALE : collocation_transparente

Explications des tests menant à la catégorie finale :
CRAN : NON
  Explication : Explication : Les mots « système » et « phonologique » sont autonomes et productifs en français.
MORPH : NON
  Explication : Explication : Le pluriel « systèmes phonologiques » reste grammatical et naturel.
SYNT : NON
  Explication : Explication : L’ordre inverse « phonologique du système » reste compréhensible et grammatical.
LEX : NON
  Explication : Explication : « système phonologique » peut être remplacé par « système des sons » sans bizarrerie.
MODIF : NON
  Explication : Explication : L’adjectif « phonologique » modifie normalement « système » ici.
SEM-REST : OUI
  Explication : Explication : « système » combine avec des ensembles typés mais limités (phonologi

Annotation des expressions nominales: 100%|██████████| 100/100 [13:15<00:00,  7.96s/expr]

Annotated and saved annotated_nominal_part_7_gpt51_single_run.xlsx
